In [1]:
from pyspark.sql import SparkSession
import inspect

In [2]:
spark = SparkSession.builder.appName('Aula de Spark').master("spark://spark-master:7077").enableHiveSupport().getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/22 02:23:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

# ACESSO HADOOP

In [4]:
df_hdfs = spark.read.csv(
    path='hdfs://namenode:8020/bronze/reddit',
    header=True
)

In [5]:
df_hdfs.show()

+-------+--------------------+-----+------------+--------------------+--------------------+--------------------+------------+------------+
|     id|               title|score|   subreddit|              author|                 url|           permalink| created_utc|num_comments|
+-------+--------------------+-----+------------+--------------------+--------------------+--------------------+------------+------------+
|1mqcpec|BF vets after pla...|    2| Battlefield|             Tintn00|https://i.redd.it...|/r/Battlefield/co...|1755204356.0|           0|
|1mqcog8| Search and Destroy?|    0| Battlefield|  Electronic-Tap3979|https://www.reddi...|/r/Battlefield/co...|1755204301.0|           0|
|1mqcnnx|As somebody who h...|    2| Battlefield|               Jon2D|https://www.reddi...|/r/Battlefield/co...|1755204252.0|           0|
|1mqcnis|        bf6 beta ps5|    0| Battlefield|Commercial_Cold_6970|https://www.reddi...|/r/Battlefield/co...|1755204242.0|           0|
|1mqcmn3|Another large map.

# ACESSO S3

In [6]:
path_bronze_reddit = "s3a://bronze/reddit"

df_bronze_reddit = spark.read.option("header", "true").csv(path_bronze_reddit)

25/08/22 02:24:04 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [7]:
df_bronze_reddit.show()

+-------+--------------------+-----+------------+--------------------+--------------------+--------------------+------------+------------+
|     id|               title|score|   subreddit|              author|                 url|           permalink| created_utc|num_comments|
+-------+--------------------+-----+------------+--------------------+--------------------+--------------------+------------+------------+
|1mqcpec|BF vets after pla...|    2| Battlefield|             Tintn00|https://i.redd.it...|/r/Battlefield/co...|1755204356.0|           0|
|1mqcog8| Search and Destroy?|    0| Battlefield|  Electronic-Tap3979|https://www.reddi...|/r/Battlefield/co...|1755204301.0|           0|
|1mqcnnx|As somebody who h...|    2| Battlefield|               Jon2D|https://www.reddi...|/r/Battlefield/co...|1755204252.0|           0|
|1mqcnis|        bf6 beta ps5|    0| Battlefield|Commercial_Cold_6970|https://www.reddi...|/r/Battlefield/co...|1755204242.0|           0|
|1mqcmn3|Another large map.

# ACESSO ODBC

In [ ]:
conn = 'jdbc:postgresql://postgres:5432/dvdrental'

In [ ]:
prop = {
    "user":"admin",
    "password":"admin",
    "driver":"org.postgresql.Driver"
}

In [ ]:
prop

In [ ]:
df_city = spark.read.jdbc(url=conn,properties=prop,table='public.city')

In [ ]:
df_city.show()

In [ ]:
df_city.count()

# QUERY NO ODBC

In [ ]:
query = '(select c.customer_id, c.first_name, c.email, c2.city \
from public.customer c \
inner join public.address a on c.address_id = a.address_id \
inner join public.city c2 on c2.city_id = a.city_id) as tab'

In [ ]:
df_query = spark.read.jdbc(url=conn,properties=prop, table=query)

In [ ]:
df_query.show(truncate=False)

In [ ]:
df_query.count()

In [ ]:
df_query.show(2)

# GRAVANDO NO DATALAKE

In [ ]:
df_query.write.csv('hdfs:///datalake/bronze/aula4', header = True, sep=';', mode='append')

In [ ]:
df_query2 = df_query.repartition(3)

In [ ]:
df_query2.write.csv('hdfs:///datalake/bronze/aula4/gravacao', header = True, sep=';', mode='overwrite')

In [ ]:
df_query2.write.json('s3a://camada-bronze/aula4/gravacao', mode='append')

In [ ]:
df_query2.write.parquet('s3a://camada-bronze/aula4/gravacao', mode='append')

# GRAVANDO NO HIVE

In [ ]:
df_query.write.format('hive').saveAsTable('default.query')

In [ ]:
df_query.write.format('hive').insertInto('default.query')

In [ ]:
df_query.write.mode('overwrite').format('hive').insertInto('default.query')

In [ ]:
# OBS AO GRAVAR NO HIVE VC NAO ESCOLHE ONDE A TABELA É SALVA, 
# RECOMENDA-SE CRIAR UMA TABELA COM A FONTE ANTES
# SPARK SO PARA POPULAR E NAO CRIAR

In [ ]:
# LENDO HIVE

In [ ]:
df_hive = spark.read.table('aula.tabelaexterna')

In [ ]:
df_hive.show()

In [ ]:
df_hive.count()

In [ ]:
df_hive = spark.read.table('default.query')

In [ ]:
# cruzando tabela hive com tabela s3

In [ ]:
df_hive.join(df_query, df_query.customer_id == df_hive.customer_id).show()

In [ ]:
df_total = spark.sql('select count(*) from parquet.`s3a://camada-bronze/aula4/gravacao`')

In [ ]:
df_total.show()

In [ ]:
df_total.write.jdbc(url=conn, properties=prop, table = 'public.aula4')

# APIs

In [ ]:
import requests

In [ ]:
retorno = requests.get('https://catfact.ninja/fact')

In [ ]:
retorno.status_code

In [ ]:
retorno.text

In [ ]:
l = []
for x in range (10):
    print(x)
    retorno = requests.get('https://catfact.ninja/fact')
    l.append(retorno.json())
    

In [ ]:
l

In [ ]:
req = spark.read.json(spark.sparkContext.parallelize(l))

In [ ]:
req.show()